# Sizing scenarios and design drivers

*Written by Marc Budinger (INSA Toulouse), Scott Delbecq (ISAE-SUPAERO) and Félix Pollet (ISAE-SUPAERO), Toulouse, France.*

The preliminary sizing of UAVs aims at defining the main drone characteristics and selecting the components based on a few **sizing scenarios**. The sizing scenarios are part of the specification process and may include both transient and endurance criteria, such as maximum rate of climb and hover flight endurance. The final overall design has to consider simultaneously the various sizing scenarios that may impact the components selection through **design drivers**.

### Design drivers

Design drivers are physical factors that influence the design process of a component. For example, the propeller design is driven, among others, by the maximum thrust it can provide.

Table 1 summarizes the main system components. 

### Sizing scenarios

Endurance scenarios, like hover flight need to be taken into account for the selection of the battery endurance and the evaluation of the motor temperature rise. Extreme performance criteria, such as maximum transitory acceleration or maximum rate of climb are fundamental to determine the maximum rotational speed, maximum propeller torque, Electronic Speed Controller (ESC) corner power, battery voltage and power, and mechanical stress. 

**<center>Table 1: Design drivers and sizing scenarios</center>**

| Components                  | Design drivers   | Parameters involved         | Sizing scenarios |
|-----------------------------|------------------|-----------------------------|------------------|
| Propeller                   	| Max thrust       	| Diameter, tip speed         	| Takeoff          	|
|                             	|                  	|                             	| Climb            	|
|                             	| Efficiency       	| Pitch                       	| Hover            	|
|                             	|                  	|                             	| Cruise           	|
| Motor                       	| Temperature rise 	| Nominal torque, Resistance  	| Hover            	|
|                             	|                  	|                             	| Cruise           	|
|                             	| Max. voltage     	| Torque constant, Resistance 	| Takeoff          	|
|                             	|                  	|                             	| Climb            	|
| Electronic Speed Controller 	| Temperature rise 	| Max power, Max current      	| Takeoff          	|
|                             	|                  	|                             	| Climb            	|
| Battery                     	| Energy           	| Voltage, Capacity           	| Hover (autonomy) 	|
|                             	|                  	|                             	| Cruise (range)   	|
|                             	| Power            	| Voltage, Max discharge rate 	| Takeoff          	|
|                             	|                  	|                             	| Climb            	|
| Frame                       	| Stress           	| Mechanical strength         	| Takeoff          	|
|                             	|                  	|                             	| Landing          	|
|                             	|                  	|                             	| Crash            	|


# Equations of sizing scenarios

The purpose of this notebook is to illustrate on an existing drone example the main equations to be taken into account during the sizing. The selected drone is the [MK-Quadro](http://wiki.mikrokopter.de/MK-Quadro) drone of Mikrokopter. The technical informations about the MK-quadro drone can be found on the website of [MikroKopter](http://wiki.mikrokopter.de/en/FlightTime) company.


```{figure} https://raw.githubusercontent.com/SizingLab/sizing_course/main/laboratories/Lab-package_delivery_UAV/assets/images/MikroQuadro.JPG
:name: apc_mr_prop
:align: center
:width: 25%

APC MR (Multi-Rotor) propellers
```


### Propellers fundamental equations

Propellers can be described in an ideal manner by the disk momentum theory. The [dimensional analysis]([https://web.mit.edu/16.unified/www/FALL/thermodynamics/notes/node86.html#SECTION06374000000000000000]) theory enables to describe with more details the propeller performances thanks to  aerodynamic coefficients:
- The thrust: $T = C_{T,h} \rho \Omega^2 A R^2=C_{T,p} \rho n^2 D^4$  which gives $ = k_{TD} \Omega^2$ for a given propeller and a given diameter
- The power: $P = C_{P,h} \rho \Omega^3 A R^3=C_{P,p} \rho n^3 D^5 $ which gives the power $P=k_{PD}\Omega^3$ or the torque $M = k_{PD} \Omega^2$ for a given propeller and a given diameter

```{note}
Be carefull, litterature shows 2 notations :
* the propeller notation with n the propeller speed in [rev/s], D the diameter in [m], $\rho$  the air density in [$kg/m^3$]
* the helicopter notation with $\Omega$ the propeller speed in [rad/s], R the radius in [m], $A$ the disk area in [$m^2$], $\rho$  the air density in [$kg/m^3$]

```

```{exercise}
:label: sizing_scenarios_equations

Compare the caracteristics of the following [APC propellers](https://www.apcprop.com/files/PER3_10x45MR.dat) to those of the MikroKopter MK-Quadro ([GWS](https://gwsus.com/english/dealer/dealer_gwsusa.htm))
```

Characteristics  | APC Propeller | GWS Propeller
-- | -- | --
Diameter [inch] | 10 | 10 
Pitch [inch] | 4.5 | 4.5 
Thrust coef $C_t$ [-] | 0.1125 | 
Power coef $C_p$ [-]  | 0.0441 | 
Thrust coef $k_{TD}$ [$\frac{N}{(rad/s)^2}$] | | $1.799.10^{-5}$
Torque coefficient $k_{PD}$ [$\frac{N}{(rad/s)^2}$] | | $3.054.10^{-7}$

with: $1$ $inch = 0.0254$ $m$, $\rho=1.18$ $kg/m^3$ 


In [35]:
D = 10 * 0.0254  # [m] Propeller diameter
rho = 1.18  # [kg/m^3] Air mass volumic (25°C)

# APC coefficients
# Ct=T/(rho * n**2 * D**4) (thrust coef.)
Ct = 0.1125  # [-] Thrust coefficient, propeller notation
# Cp=P/(rho * n**3 * D**5) (power coef.)
Cp = 0.0441  # [-] Power coefficient, propeller notation

# GWS coefficients
from math import pi

Ct_gws = 1.799e-5 * (2 * pi)**2 / (rho * D**4)
Cp_gws = 3.054e-7 * (2 * pi)**3 / (rho * D**5)

factor_apc = Ct/Cp
factor_gws = Ct_gws/Cp_gws

print(f"GWS Ct: {Ct_gws}")
print(f"GWS Cp: {Cp_gws}")

print(f"APC factor: {factor_apc}")
print(f"GWS factor: {factor_gws}")

GWS Ct: 0.14460190894243805
GWS Cp: 0.06072367502534727
APC factor: 2.5510204081632653
GWS factor: 2.3813102366099934


### Hover flight scenario

The flight conditions in static hover are no forward speed, no vertical speed. This scenario influences in a very important way the autonomy of the vehicle. The thrust generated by propellers have to compensate the global weight of the drone and the load.

```{figure} https://upload.wikimedia.org/wikipedia/commons/thumb/5/56/Actuator_disk.svg/1024px-Actuator_disk.svg.png
:alt: Disk Momentum Theory.
:name: fig-disk_momentum
:width: 30%

An actuator disk accelerating a fluid flow from right to left (Source: [Wikipedia](https://en.wikipedia.org/wiki/Momentum_theory)).
```


```{exercise}
:label: sizing_scenarios_propeller_thrust

With the given technical informations, calculate the requested thrust for each propeller for a no load hover flight.
```


Characteristics  | MK-MikroKopter drone
-- | -- 
Total mass ready to take off |  1350 g
Number of propellers | 4

In [15]:
# Hover scenario
# ------------
M_total = 1.35  # [kg] mass of the drone

N_pro = 4  # [-] propeller number
D_pro = 2.54e-2 * 10  # [m] propeller diameter

# to be completed
from scipy.constants import g
F_pro_hov = M_total * g / N_pro
print(f"Thrust in hovering of one propeller [N]: {F_pro_hov}")

Thrust in hovering of one propeller [N]: 3.309744375


```{exercise}
:label: sizing_scenarios_propeller_power

With the given technical informations of GWS propellers calculate the propeller torque, speed and the mechanical power.
```

In [26]:
# Propeller characteristics
# -----------------------------

K_TD = 1.799e-5  # [N/(rad/s)^2] thrust coef
K_PD = 3.054e-7  # [N.m/(rad/s)^3] torque coef

# to be compared to the ones calculated with regression
# GWS 10*4.5

# Torque and RPM of motors
# -----------------------

# to be completed
W = (F_pro_hov / K_TD)**(1/2)
P = (F_pro_hov / K_PD)**(1/3)
M = K_PD * W**2

print("----------------")
print("Propeller speed: %.2f rad/s or %.0f RPM" % (W, W / 2 / 3.14 * 60))
print("Propeller torque: %.2f N.m " % (M))
print("Propeller power: %.2f W " % (M * W))

----------------
Propeller speed: 428.93 rad/s or 4098 RPM
Propeller torque: 0.06 N.m 
Propeller power: 24.10 W 


> Questions: 

```{exercise}
:label: sizing_scenarios_autonomy

With the following technical informations calculate the motor voltage and current, the autonomy of the drone if we assume an efficiency of 95% for electronic speed controllers (ESC) and a depth of discharge of 80% for the battery.
```

Component |Characteristic | Value
--- | --- | --- 
Motor | Poles pair number| 7  
| Load current | 6‐9A (DC)  
| Max current | 10A (DC)  
| Speed constant Kv | 760 tr/min/V  
| Mechanical power | 110 W  
| Dimensions | 28.8 x 29 mm  
| Max efficiency | 76 %  
| mass | 65 g  
| Resistance | R = 0.26 ohm  
| Inertia | J= 2.5e-5 kg.m²  
Battery| Voltage | LIPO 4S (4*3.7V)
| Capacity | 3300mAh
| mass | 329g


In [27]:
# Voltage / current calculation
# -----------------------------

K_T_mot = (1 / 760) * 60 / 2 / 3.14  # [V/(rad/s)] Motor torque coef
R_mot = 0.26  # [Ohm] Motor resistance

# Voltage / current calculation

# to be completed
I = M / K_T_mot
U = W * K_T_mot + R_mot * I

print("----------------")
print("Motor current: %.2f A " % I)
print("Motor voltage: %.2f V " % U)

# Autonomy of the battery

# to be completed
V_bat = 4 * 3.7
I_bat = (U * I * N_pro) / 0.95 / V_bat
Aut_bat = 3.3 * 0.8 * 60 / I_bat 

print("----------------")
print("Battery current: %.2f A " % I_bat)
print("Battery voltage: %.2f V " % V_bat)
print("Autonomy of the battery : %.1f min" % Aut_bat)

----------------
Motor current: 4.47 A 
Motor voltage: 6.55 V 
----------------
Battery current: 8.33 A 
Battery voltage: 14.80 V 
Autonomy of the battery : 19.0 min
